In [3]:
# prepares data for the baseline model and transfer learning models

import os, sys
import cv2
import numpy as np

in_path = 'd'

i = 0
x_train = []
y_train = []
x_test = []
y_test = []
label = {}
for dir in os.listdir(in_path):
    path = os.path.join(in_path, dir)
    if not os.path.isdir(path): continue
    fns = os.listdir(path)
    size = len(fns)
    train_size = int(size * 0.8)
    test_size = size - train_size
    imgs = []
    for fn in fns:
        img = cv2.imread(os.path.join(path, fn))
        if img is None: 
            size -= 1
            train_size = int(size * 0.8)
            test_size = size - train_size
            continue
        imgs.append(img)
    imgs = np.array(imgs)
    train_indices = np.random.choice(size, train_size, replace = False)
    test_indices = np.ones(size, np.bool)
    test_indices[train_indices] = 0
    x_train.extend(imgs[train_indices])
    y_train += train_size * [i]
    y_test += test_size * [i]
    xt = imgs[test_indices]
    x_test.extend(xt)
    yt = np.array(test_size * [i])
    print(xt.shape, yt.shape)
    t = {'x_test': xt, 'y_test': yt}
    np.savez('data/' + dir + '.npz', **t)
    label[i] = dir
    i += 1


i=np.random.permutation(len(x_train))
x_train = np.array(x_train)[i]
y_train = np.array(y_train)[i]
x_test = np.array(x_test)
y_test = np.array(y_test)

data = {'x_train': x_train, 'y_train': y_train, 'x_test': x_test, 'y_test': y_test, 'label': label}
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)
np.savez('d.npz', **data)

(60, 224, 224, 3) (60,)
(66, 224, 224, 3) (66,)
(89, 224, 224, 3) (89,)
(55, 224, 224, 3) (55,)
(74, 224, 224, 3) (74,)
(68, 224, 224, 3) (68,)
(97, 224, 224, 3) (97,)
(64, 224, 224, 3) (64,)
(61, 224, 224, 3) (61,)
(134, 224, 224, 3) (134,)
(64, 224, 224, 3) (64,)
(82, 224, 224, 3) (82,)
(132, 224, 224, 3) (132,)
(67, 224, 224, 3) (67,)
(76, 224, 224, 3) (76,)
(63, 224, 224, 3) (63,)
(78, 224, 224, 3) (78,)
(69, 224, 224, 3) (69,)
(68, 224, 224, 3) (68,)
(89, 224, 224, 3) (89,)
(62, 224, 224, 3) (62,)
(62, 224, 224, 3) (62,)
(90, 224, 224, 3) (90,)
(7037, 224, 224, 3) (7037,) (1770, 224, 224, 3) (1770,)


In [4]:
data['label']

{0: 'ando',
 1: 'coop',
 2: 'corb',
 3: 'eisenman',
 4: 'gehry',
 5: 'holl',
 6: 'ito',
 7: 'kahn',
 8: 'koolhaas',
 9: 'libeskind',
 10: 'mayne',
 11: 'meier',
 12: 'mvrdv',
 13: 'oscar',
 14: 'pei',
 15: 'renzo',
 16: 'rogers',
 17: 'rohe',
 18: 'sanaa',
 19: 'tschumi',
 20: 'wang',
 21: 'wright',
 22: 'zaha'}

In [1]:
# prepares data for the one-shot learning model

import os, sys
import cv2
import numpy as np

tr = 174
va = 44
te = 55
x_train = []
y_train = []
x_val = []
y_val = []
x_test = []
y_test = []

for i in range(23):
    x_train.append([])
    x_val.append([])
    x_test.append([])

data = np.load('d.npz', allow_pickle = True)
data = dict(data)
xt = data['x_train']
yt = data['y_train']
xv = data['x_test']
yv = data['y_test']

c1 = np.zeros(23, dtype = int)
for i in range(7037):
    c = yt[i]
    if c1[c] < tr:
        x_train[c].append(xt[i])
    else: continue
    c1[c] += 1

c1 = np.zeros(23, dtype = int)
for i in range(7036, -1, -1):
    c = yt[i]
    if c1[c] < va:
        x_val[c].append(xt[i])
    else: continue   
    c1[c] += 1

c1 = np.zeros(23, dtype = int)
for i in range(1770):
    c = yv[i]
    if c1[c] < te:
        x_test[c].append(xv[i])
    else: continue
    c1[c] += 1
    
for i in range(23):
    y_train.extend([i] * tr)
    y_val.extend([i] * va)
    y_test.extend([i] * te)
    
for i in range(23):
    for j in range(10):
        temp = x_train[i][j]
        x_train[i][j] = x_val[i][j]
        x_val[i][j] = temp
    
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
x_val = np.array(x_val)
y_val = np.array(y_val)
    
data = {'x_train': x_train, 'y_train': y_train, 'x_val': x_val, 'y_val': y_val, 'x_test': x_test, 'y_test': y_test}
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_val.shape, y_val.shape)
np.savez('df6.npz', **data)

(23, 174, 224, 224, 3) (4002,) (23, 55, 224, 224, 3) (1265,) (23, 44, 224, 224, 3) (1012,)
